In [29]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [9]:
match_wise_df = pd.read_csv("../../../data/fivb/2019_women_world_cup_match-wise.csv")
player_wise_df = pd.read_csv("../../../data/fivb/2019_women_world_cup_player-wise.csv")
matches_df = pd.read_csv("../../../data/fivb/2019_women_world_cup_matches.csv")

In [10]:
match_wise_df.head(20)

,Match Num,Team,Opponent,Set 1 Duration,Set 1 Scored,Set 1 Against,Set 2 Duration,Set 2 Scored,Set 1 Against.1,Set 3 Duration,Set 3 Scored,Set 3 Against,Attack,Block,Serve,Opp. Errors,Total,Dig,Reception,Set
0,1,USA,KEN,19,25,20,24,25,20,20,25,14,47,6,7,15,75,40,17,31
1,1,KEN,USA,19,14,25,24,20,25,20,14,25,29,4,2,13,48,19,11,13
2,2,CMR,RUS,21,14,25,26,15,25,21,10,25,20,6,1,12,39,22,6,15
3,2,RUS,CMR,21,25,15,26,25,15,21,25,10,39,9,8,19,75,40,5,14
4,3,ARG,NED,21,16,25,26,17,25,23,19,25,35,2,2,13,52,42,16,6
5,3,NED,ARG,21,25,17,26,25,17,23,25,19,47,8,4,16,75,43,15,14
6,4,CHN,KOR,25,25,15,23,25,15,22,25,14,46,12,6,11,75,53,22,24
7,4,KOR,CHN,25,21,25,23,15,25,22,14,25,37,4,2,7,50,45,14,14
8,5,SRB,BRA,28,20,23,29,25,23,26,18,25,58,13,5,24,100,81,17,25
9,5,BRA,SRB,28,25,25,29,23,25,26,25,18,64,18,3,25,110,78,19,51


In [11]:
player_wise_df

,Match Num,Team,Number,Name,Points Scored,Attacks Won,Attacks Attempted,Blocks Won,Blocks Attempted,Serves Won,Serves Attempted
0,1,USA,11,Andrea Drews,15,12,22,1,12,2,10
1,1,USA,10,Jordan Larson,14,10,23,1,3,3,15
2,1,USA,24,Chiaka Ogbogu,12,9,11,3,15,0,9
3,1,USA,23,Kelsey Robinson,11,10,19,0,1,1,10
4,1,USA,22,Haleigh Washington,3,3,5,0,4,0,8
...,...,...,...,...,...,...,...,...,...,...,...
1840,66,JPN,2,Sarina Koga,0,0,0,0,0,0,0
1841,66,JPN,3,Nana Iwasaka,0,0,0,0,0,0,0
1842,66,JPN,5,Erika Shinomiya,0,0,0,0,0,0,0
1843,66,JPN,14,Mako Kobata,0,0,0,0,0,0,0


In [12]:
df = match_wise_df
features = ["Attack", "Block", "Serve", "Opp. Errors", "Total", "Dig", "Reception", "Set"]
raw_df = df[features]
cum_df = pd.DataFrame(np.zeros(raw_df.shape), columns=features)
for i in range(len(cum_df)):
    team = df.loc[i]["Team"]
    count = 1
    for j in range(i-1, -1, -1):
        if df.loc[j]["Team"] == team:
            cum_df.loc[i] += raw_df.loc[j]
            count += 1
    cum_df.loc[i] /= count
cum_df


,Attack,Block,Serve,Opp. Errors,Total,Dig,Reception,Set
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
127,42.000000,5.000000,3.454545,12.545455,63.000000,50.090909,14.454545,22.636364
128,32.272727,4.909091,3.181818,12.818182,53.181818,36.363636,10.909091,19.000000
129,28.454545,3.818182,2.545455,10.636364,45.454545,29.909091,9.727273,18.818182
130,50.636364,4.545455,3.818182,15.272727,74.272727,64.000000,20.727273,31.545455


In [13]:
matches_df

,Match Num,TeamA,TeamB,TeamA Score,TeamB Score,Winner
0,1,USA,KEN,3,0,0
1,2,CMR,RUS,0,3,1
2,3,ARG,NED,0,3,1
3,4,CHN,KOR,3,0,0
4,5,SRB,BRA,2,3,1
...,...,...,...,...,...,...
61,62,DOM,SRB,3,1,0
62,63,RUS,BRA,1,3,1
63,64,CHN,ARG,3,0,0
64,65,CMR,KEN,1,3,1


In [14]:
features = np.zeros((len(match_wise_df) // 2, 2*8))
for i in range(len(match_wise_df) // 2):
    features[i][:8] = cum_df.loc[2*i]
    features[i][8:] = cum_df.loc[2*i + 1]

labels = matches_df["Winner"].to_numpy()

print(features.shape, labels.shape)


(66, 16) (66,)


In [39]:
models = [
    XGBClassifier(),
    MLPClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
]
result_dict = {"Model": [], "Accuracy Mean": [], "Accuracy Dev": []}
for m in models:
    scores = cross_val_score(m, features, labels, scoring="accuracy")
    result_dict["Model"].append(m.__class__.__name__)
    result_dict["Accuracy Mean"].append(scores.mean())
    result_dict["Accuracy Dev"].append(scores.std())

results_df = pd.DataFrame(result_dict)
print(results_df)

                    Model  Accuracy Mean  Accuracy Dev
0           XGBClassifier       0.667033      0.132672
1           MLPClassifier       0.726374      0.135410
2  RandomForestClassifier       0.682418      0.139019
3      LogisticRegression       0.740659      0.081526
4                     SVC       0.605495      0.091308
5  DecisionTreeClassifier       0.529670      0.165406
